<a href="https://colab.research.google.com/github/MythEclipse/Belajar-MachineLearning/blob/main/proyek_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
!kaggle datasets download -d lorenzovzquez/data-jobs-salaries

  0% 0.00/66.5k [00:00<?, ?B/s]
100% 66.5k/66.5k [00:00<00:00, 45.2MB/s]


In [ ]:
!unzip data-jobs-salaries.zip

Archive:  data-jobs-salaries.zip
  inflating: salaries.csv            


In [ ]:
import tensorflow as tf
import urllib.request
import zipfile
import re
import os
from tensorflow.keras.utils import get_file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.layers import Bidirectional,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,GRU,GlobalMaxPooling1D,BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [ ]:
df = pd.read_csv('salaries.csv')

In [ ]:
print(df.info())
print(df.head())
print(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10417 entries, 0 to 10416
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           10417 non-null  int64 
 1   experience_level    10417 non-null  object
 2   employment_type     10417 non-null  object
 3   job_title           10417 non-null  object
 4   salary              10417 non-null  int64 
 5   salary_currency     10417 non-null  object
 6   salary_in_usd       10417 non-null  int64 
 7   employee_residence  10417 non-null  object
 8   remote_ratio        10417 non-null  int64 
 9   company_location    10417 non-null  object
 10  company_size        10417 non-null  object
dtypes: int64(4), object(7)
memory usage: 895.3+ KB
None
   work_year experience_level employment_type                  job_title  \
0       2023               SE              FT            Prompt Engineer   
1       2023               SE              FT            P

In [ ]:
time_series_data = df[['work_year', 'salary_in_usd']]

In [ ]:
scaler = MinMaxScaler()
time_series_data = time_series_data.copy()
time_series_data.loc[:, 'scaled_salary'] = scaler.fit_transform(time_series_data['salary_in_usd'].values.reshape(-1, 1))


In [ ]:
sequence_length = 10
sequences = []
for i in range(len(time_series_data) - sequence_length):
    sequence = time_series_data['scaled_salary'].values[i:i+sequence_length]
    target = time_series_data['scaled_salary'].values[i+sequence_length]
    sequences.append((sequence, target))


In [ ]:
X = np.array([sequence for sequence, _ in sequences])
y = np.array([target for _, target in sequences])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val_reshaped = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(units=100, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(units=1))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

In [ ]:
callbacks = EarlyStopping(monitor='val_mae', patience=5, mode='min', min_delta=0.1)

In [ ]:
history=model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_val_reshaped, y_val),callbacks=[callbacks])

Epoch 1/10
261/261 [==============================] - 7s 18ms/step - loss: 0.0087 - mae: 0.0718 - val_loss: 0.0067 - val_mae: 0.0651
Epoch 2/10
261/261 [==============================] - 4s 17ms/step - loss: 0.0081 - mae: 0.0693 - val_loss: 0.0068 - val_mae: 0.0657
Epoch 3/10
261/261 [==============================] - 4s 14ms/step - loss: 0.0080 - mae: 0.0692 - val_loss: 0.0066 - val_mae: 0.0646
Epoch 4/10
261/261 [==============================] - 3s 12ms/step - loss: 0.0080 - mae: 0.0688 - val_loss: 0.0071 - val_mae: 0.0680
Epoch 5/10
261/261 [==============================] - 3s 11ms/step - loss: 0.0079 - mae: 0.0686 - val_loss: 0.0070 - val_mae: 0.0675
Epoch 6/10
261/261 [==============================] - 3s 13ms/step - loss: 0.0079 - mae: 0.0687 - val_loss: 0.0066 - val_mae: 0.0641


In [ ]:
y_val_pred = model.predict(X_val_reshaped)

66/66 [==============================] - 1s 4ms/step


In [ ]:
mae = mean_absolute_error(y_val, y_val_pred)

In [ ]:
print("Training MAE:", history.history['mae'])
print("Validation MAE:", history.history['val_mae'])

Training MAE: [0.07183659821748734, 0.0692858025431633, 0.06915853172540665, 0.06882474571466446, 0.06855069100856781, 0.0686761736869812]
Validation MAE: [0.06511033326387405, 0.06572798639535904, 0.06456683576107025, 0.0680491030216217, 0.0674966350197792, 0.06411073356866837]


In [ ]:
if history.history['val_mae'][-1] < 0.1:
    print("berhasil")
else:
    print("gagal")

berhasil
